In [1]:
from __future__ import print_function

Nastavime parametry ze zadani. Mel jsem problem s nastavenim sveho vlastniho modulu v telesu, ale sage si defaultne zvolil ten ze zadani.

In [2]:
p = 2
n = 47
mod = 'x^47 + x^5 + 1'
# G = GF(p^n, name = 'x', modulus=mod)
G = GF(p^n, name = 'x')
print('Mame teleso:', G, 'a v nem modulo:', G.modulus())

Mame teleso: Finite Field in x of size 2^47 a v nem modulo: x^47 + x^5 + 1


1. Faze algoritmu.
Nastavime mnozinu S ireducibilnimi polynomy.

In [3]:
R.<x> = PolynomialRing(Integers(2))
s = {}
max_polynomial = G.gen()
# zde nastavime max polynomu
max_deg = 7
cnt = 0
for i in R.polynomials(max_degree=max_deg):
    if i.is_irreducible():
        s[i] = cnt
        cnt += 1
        max_polynomial = i
#         print(i)
print('Mame', len(s), 'ireducibilnich polynomu stupne max.', max_deg)

Mame 41 ireducibilnich polynomu stupne max. 7


Nastavime promene pro cyklus.

In [4]:
soustava = []
vecs = []

2. Faze

Hledam nahodne vhodna l, pro ktera plati, ze x^l se da faktorizovat na nase ireducibilni polynomy v mnozine S.

Sage nam pomerne rychle faktorizuje polynomy. My se pouze podivame na nejvyssi z nich a pokud je mensi nez nas nejvyssi ireducibilni polynom, pridame ho do soustavy.

In [5]:

while len(soustava) < len(s):
    l = floor(random()*(p^n))
    if G(max_polynomial) >= G(G.polynomial_ring()((G.gen()**l)).factor()[-1][0]):
        rovnice = [l, G.polynomial_ring()((G.gen()**l)).factor()]
        soustava.append(rovnice)
#         print(G.polynomial_ring()((G.gen()**l)))
#         for ii in G.polynomial_ring()((G.gen()**l)).factor():
#             print(ii[0])
#             print(s[ii[0]])

Polynomy pro ktere mame vyresit DLP.

In [6]:
h1 = G(x^45 + x^44 + x^43 + x^39 + x^38 + x^37 + x^34 + x^31 + x^28 + x^24 + x^20 + x^19 + x^12 + x^11 + x^8 + x^7 + x^2 + 1)
h2 = G(x^46 + x^45 + x^38 + x^31 + x^30 + x^29 + x^25 + x^24 + x^22 + x^18 + x^17 + x^14 + x^9 + x^7 + x)

3. Faze

Soustavu rovnic zapisu do matice.

In [7]:
vecs = []
for ii in soustava:
    vec = [0] * (len(s))
    for i in ii[1]:
        vec[s[i[0]]] = i[1]
    vecs.append(vec)

M = IntegerModRing(G.order()-1)
# print(len(vecs))
matice = Matrix(M,vecs)

Vytvorime vektor prave strany.

In [8]:
prava_strana = []
for i in soustava:
    prava_strana.append(i[0])

Vyresime soustavu rovnic.

In [9]:

X = matice.solve_right(vector(prava_strana))

4. Faze

Hledam l, pro ktere plati, ze h*g^-l se da faktorizovat na nase ireducibilni polynomy. Opet vyuzijeme sage pro faktorizaci.

In [11]:
for i in range(100000):
    l = floor(random()*(p^n))
    
    if G(max_polynomial) >= G(G.polynomial_ring()(h2*(G.gen()**(-l))).factor()[-1][0]):
        print(l)
        break

80095213282333


Spocitame DLP.

In [12]:
cislo = 0
for i in G.polynomial_ring()(h2*(G.gen()**(-l))).factor():
    cislo += (X[s[i[0]]]*i[1])
cislo += l
cislo = Mod(cislo, G.order()-1) 
print(cislo)

125832502466259


Podivame se, zdali jsme nasli dobre reseni.

In [13]:
G.gen()**cislo == h2

True

Vidime, ze ano, totez udelame pro druhe h.

In [14]:
for i in range(100000):
    l = floor(random()*(p^n))
    
    if G(max_polynomial) >= G(G.polynomial_ring()(h1*(G.gen()**(-l))).factor()[-1][0]):
        print(l)
        break

16754202874732


In [15]:
cislo = 0
for i in G.polynomial_ring()(h1*(G.gen()**(-l))).factor():
    cislo += (X[s[i[0]]]*i[1])
cislo += l
cislo = Mod(cislo, G.order()-1) 
print(cislo)

136914677089945


In [16]:
G.gen()**cislo == h1

True

V reseni se jeste muzeme utvrdit pouzitim sage dlp, ktere nam vrati stejne vysledky jako algoritmus vyse pouze v mnohem kratsi case (Podle dokumentace vyuziva Pohlig–Hellman a Baby-step giant-step).

In [18]:
print(discrete_log(h2, G.gen(), G.order()-1))
print(discrete_log(h1, G.gen(), G.order()-1))

125832502466259
136914677089945
